<a href="https://colab.research.google.com/github/NIMZB/Predicting-chemical-properties-of-drugs/blob/master/property_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import csv
# import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET
import numpy as np

import requests
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
% ls

full.xml  Untitled0.ipynb


In [0]:
# xml_path = os.path.join('download', 'drugbank.xml.gz')
# with gzip.open(xml_path) as xml_file:
xml_file = '/content/drive/My Drive/Genomics/full.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

In [0]:
ns = '{http://www.drugbank.ca}'
for i,drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'

In [0]:
ns = '{http://www.drugbank.ca}'
exp_mw_template = "{ns}experimental-properties/{ns}property[{ns}kind='Molecular Weight']/{ns}value"
exp_logp_template = "{ns}experimental-properties/{ns}property[{ns}kind='logP']/{ns}value"
exp_pka_template = "{ns}experimental-properties/{ns}property[{ns}kind='pKa']/{ns}value"
exp_mp_template = "{ns}experimental-properties/{ns}property[{ns}kind='Melting Point']/{ns}value"
exp_bp_template = "{ns}experimental-properties/{ns}property[{ns}kind='Boiling Point']/{ns}value"
exp_ws_template = "{ns}experimental-properties/{ns}property[{ns}kind='Water Solubility']/{ns}value"
exp_logs_template = "{ns}experimental-properties/{ns}property[{ns}kind='logS']/{ns}value"
exp_hydro_template = "{ns}experimental-properties/{ns}property[{ns}kind='Hydrophobicity']/{ns}value"
exp_ip_template = "{ns}experimental-properties/{ns}property[{ns}kind='Isoelectronic Point']/{ns}value"
exp_cp_template = "{ns}experimental-properties/{ns}property[{ns}kind='caco2 Permeability']/{ns}value"
exp_rad_template = "{ns}experimental-properties/{ns}property[{ns}kind='Radioactivity']/{ns}value"
cal_logp_template = "{ns}calculated-properties/{ns}property[{ns}kind='logP']/{ns}value"
cal_logs_template = "{ns}calculated-properties/{ns}property[{ns}kind='logS']/{ns}value"
cal_ws_template = "{ns}calculated-properties/{ns}property[{ns}kind='Water Solubility']/{ns}value"
cal_mw_template = "{ns}calculated-properties/{ns}property[{ns}kind='Molecular Weight']/{ns}value"
cal_psa_template = "{ns}calculated-properties/{ns}property[{ns}kind='Polar Surface Area (PSA)']/{ns}value"
cal_refr_template = "{ns}calculated-properties/{ns}property[{ns}kind='Refractivity']/{ns}value"
cal_pola_template = "{ns}calculated-properties/{ns}property[{ns}kind='Polarizability']/{ns}value"
cal_rbc_template = "{ns}calculated-properties/{ns}property[{ns}kind='Rotatable Bond Count']/{ns}value"
cal_hdac_template = "{ns}calculated-properties/{ns}property[{ns}kind='H Bond Acceptor Count']/{ns}value"
cal_hbdc_template = "{ns}calculated-properties/{ns}property[{ns}kind='H Bond Donor Count']/{ns}value"
cal_pkasa_template = "{ns}calculated-properties/{ns}property[{ns}kind='pKa (strongest acidic)']/{ns}value"
cal_pkasb_template = "{ns}calculated-properties/{ns}property[{ns}kind='pKa (strongest basic)']/{ns}value"
cal_pc_template = "{ns}calculated-properties/{ns}property[{ns}kind='Physiological Charge']/{ns}value"
cal_nr_template = "{ns}calculated-properties/{ns}property[{ns}kind='Number of Rings']/{ns}value"
cal_bioa_template = "{ns}calculated-properties/{ns}property[{ns}kind='Bioavailability']/{ns}value"
cal_rof_template = "{ns}calculated-properties/{ns}property[{ns}kind='Rule of Five']/{ns}value"

In [0]:
rows = list()

for i, drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'
    # row['type'] = drug.get('type')
    # row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
    # row['name'] = drug.findtext(ns + "name")
    row['cal_logp'] = drug.findtext(cal_logp_template.format(ns = ns))
    row['cal_logs'] = drug.findtext(cal_logs_template.format(ns = ns))
    row['cal_water_solubility'] = drug.findtext(cal_ws_template.format(ns = ns))
    row['cal_molecular_weight'] = drug.findtext(cal_mw_template.format(ns = ns))
    row['cal_polar_surface_area'] = drug.findtext(cal_psa_template.format(ns = ns))
    row['cal_refractivity'] = drug.findtext(cal_refr_template.format(ns = ns))
    row['cal_polarizability'] = drug.findtext(cal_pola_template.format(ns = ns))
    row['cal_rotatable_bond_count'] = drug.findtext(cal_rbc_template.format(ns = ns))
    row['cal_h_bond_acceptor_count'] = drug.findtext(cal_hdac_template.format(ns = ns))
    row['cal_h_bond_donor_count'] = drug.findtext(cal_hbdc_template.format(ns = ns))
    row['cal_pka_acidic'] = drug.findtext(cal_pkasa_template.format(ns = ns))
    row['cal_pka_basic'] = drug.findtext(cal_pkasb_template.format(ns = ns))
    row['cal_physiological_charge'] = drug.findtext(cal_pc_template.format(ns = ns))
    row['cal_number_of_rings'] = drug.findtext(cal_nr_template.format(ns = ns))
    row['cal_bioavailability'] = drug.findtext(cal_bioa_template.format(ns = ns))
    row['cal_rule_of_five'] = drug.findtext(cal_rof_template.format(ns = ns))
    # row['exp_molecular_weight'] = drug.findtext(exp_mw_template.format(ns = ns))
    row['exp_logp'] = drug.findtext(exp_logp_template.format(ns = ns))
    # row['exp_pka'] = drug.findtext(exp_pka_template.format(ns = ns))
    # row['exp_mp'] = drug.findtext(exp_mp_template.format(ns = ns))
    # row['exp_ws'] = drug.findtext(exp_ws_template.format(ns = ns))
    # row['exp_hydro'] = drug.findtext(exp_hydro_template.format(ns = ns))

    rows.append(row)

In [0]:
def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return rowh

rows = list(map(collapse_list_values, rows))
drugbank_df = pd.DataFrame.from_dict(rows)
drugbank_df.replace(r'^\s*$', np.nan, regex=True)
drugbank_df=drugbank_df.dropna()
drugbank_df['cal_water_solubility']=drugbank_df['cal_water_solubility'].str.replace(" g/l","")
# drugbank_df['cal_water_solubility']=pd.to_numeric(drugbank_df.cal_water_solubility)
# drugbank_df['cal_logp']=pd.to_numeric(drugbank_df.cal_logp)
# drugbank_df['cal_logs']=pd.to_numeric(drugbank_df.cal_logs)
# drugbank_df['cal_molecular_weight']=pd.to_numeric(drugbank_df.cal_molecular_weight)
# drugbank_df['cal_polar_surface_area']=pd.to_numeric(drugbank_df.cal_polar_surface_area)
# drugbank_df['cal_refractivity']=pd.to_numeric(drugbank_df.cal_refractivity)
# drugbank_df['cal_polarizability']=pd.to_numeric(drugbank_df.cal_polarizability)
# drugbank_df['cal_rotatable_bond_count']=pd.to_numeric(drugbank_df.cal_rotatable_bond_count)
# drugbank_df['cal_rotatable_bond_count']=pd.to_numeric(drugbank_df.cal_rotatable_bond_count)
drugbank_df = drugbank_df.astype(np.float64)

In [0]:
drugbank_df.to_csv(path_or_buf='/content/drive/My Drive/Genomics/data.csv', sep='\t',index=False)

In [0]:
drugbank_df.dtypes

cal_logp                     float64
cal_logs                     float64
cal_water_solubility         float64
cal_molecular_weight         float64
cal_polar_surface_area       float64
cal_refractivity             float64
cal_polarizability           float64
cal_rotatable_bond_count     float64
cal_h_bond_acceptor_count    float64
cal_h_bond_donor_count       float64
cal_pka_acidic               float64
cal_pka_basic                float64
cal_physiological_charge     float64
cal_number_of_rings          float64
cal_bioavailability          float64
cal_rule_of_five             float64
exp_logp                     float64
dtype: object

In [0]:
drugbank_df=drugbank_df.sample(frac=1)
n_samples=len(drugbank_df)
per=0.8
op=drugbank_df['cal_bioavailability']
ip=drugbank_df.drop(['cal_bioavailability'],axis=1)

ip_train=ip[:int(per*n_samples)]
op_train=op[:int(per*n_samples)]
ip_test=ip[int(per*n_samples):]
op_test=op[int(per*n_samples):]

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lreg = LogisticRegression(random_state=0, solver='liblinear',max_iter=1000 )
lreg.fit(ip_train,op_train)

predicted=lreg.predict(ip_test)
print('Explained varience score:%f'%metrics.explained_variance_score(op_test,predicted))



Explained varience score:0.773757


In [0]:
from sklearn import linear_model
from sklearn import metrics
clf = linear_model.Lasso(alpha=0.7, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=7,
   selection='cyclic', tol=0.01, warm_start=False)
clf.fit(ip_train,op_train)

predicted=clf.predict(ip_test)
print('Explained varience score:%f'%metrics.explained_variance_score(op_test,predicted))



Explained varience score:0.424062


In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
rreg = RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)
rreg.fit(ip_train,op_train)

predicted=rreg.predict(ip_test)
print('Explained varience score:%f'%metrics.explained_variance_score(op_test,predicted))



Explained varience score:0.900180


In [0]:
 from sklearn.linear_model import ElasticNet
from sklearn import metrics
regr = ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=True,
      random_state=10, selection='random', tol=0.001, warm_start=True)
regr.fit(ip_train,op_train)

predicted=regr.predict(ip_test)
print('Explained varience score:%f'%metrics.explained_variance_score(op_test,predicted))



Explained varience score:0.423602
